# Initialization

In [2]:
import logging

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

# Загрузка данных

In [4]:
items = pd.read_parquet("items.par")
events = pd.read_parquet("events.par")

In [8]:
train_test_global_time_split_date = pd.to_datetime("2017-08-01").date()

train_test_global_time_split_idx = events["started_at"] < train_test_global_time_split_date
events_train = events[train_test_global_time_split_idx]
events_test = events[~train_test_global_time_split_idx]

# Разбиение с учётом хронологии

Рекомендательные системы на практике работают с учётом хронологии. Поэтому поток событий для тренировки и валидации полезно делить на то, что уже случилось, и что ещё случится. Это позволяет проводить валидацию на тех же пользователях, на которых тренировались, но на их событиях в будущем.

# === Знакомство: "холодный" старт

In [77]:
# зададим точку разбиения
train_test_global_time_split_date = pd.to_datetime("2017-08-01").date()

train_test_global_time_split_idx = events["started_at"] < train_test_global_time_split_date
events_train = events[train_test_global_time_split_idx]
events_test = events[~train_test_global_time_split_idx]

# количество пользователей в train и test
users_train = events_train["user_id"].drop_duplicates()
users_test = events_test["user_id"].drop_duplicates()
# количество пользователей, которые есть и в train, и в test
common_users = set(users_train) & set(users_test)

print(len(users_train), len(users_test), len(common_users))

428220 123223 120858


In [78]:
cold_users = set(users_test) - set(users_train)

print(len(cold_users)) 

2365


In [79]:
top_pop_start_date = pd.to_datetime("2015-01-01").date()

item_popularity = events_train \
    .query("started_at >= @top_pop_start_date") \
    .groupby(["book_id"]).agg(users=("user_id", "nunique"), avg_rating=("rating", "mean")).reset_index()
item_popularity["popularity_weighted"] = item_popularity["users"] * item_popularity["avg_rating"]

# сортируем по убыванию взвешенной популярности
item_popularity = item_popularity.sort_values('popularity_weighted',
                                              ascending=False)

# выбираем первые 100 айтемов со средней оценкой avg_rating не меньше 4
top_k_pop_items = item_popularity.query('avg_rating>=4').iloc[:100]

In [80]:
top_k_pop_items

,book_id,users,avg_rating,popularity_weighted
32387,18007564,20207,4.321275,87320.0
32623,18143977,19462,4.290669,83505.0
30695,16096824,16770,4.301014,72128.0
2,3,15139,4.706057,71245.0
3718,38447,14611,4.232770,61845.0
...,...,...,...,...
19596,2767052,4361,4.413437,19247.0
32835,18293427,4674,4.092640,19129.0
378,3636,4667,4.098564,19128.0
33611,18966819,4361,4.374914,19079.0


In [9]:
events_test[events_test["user_id"].isin(cold_users)] 

,user_id,book_id,started_at,read_at,is_read,rating,is_reviewed,started_at_month
39766,1361610,6900,2017-10-09,2017-10-13,True,4,False,2017-10-01
39767,1361610,12555,2017-09-21,2017-10-11,True,3,False,2017-09-01
39768,1361610,25899336,2017-09-12,2017-09-17,True,4,True,2017-09-01
39769,1361610,21936809,2017-08-20,2017-08-24,True,4,True,2017-08-01
39770,1361610,6952,2017-09-18,2017-09-20,True,3,False,2017-09-01
...,...,...,...,...,...,...,...,...
12905615,1178502,252499,2017-09-30,2017-10-06,True,4,False,2017-09-01
12905624,1253160,51113,2017-09-25,2017-10-07,True,4,False,2017-09-01
12905625,1253160,16181775,2017-09-24,2017-09-25,True,3,False,2017-09-01
12905626,1253160,10210,2017-09-16,2017-09-24,True,5,False,2017-09-01


In [11]:
cold_users_events_with_recs = \
    events_test[events_test["user_id"].isin(cold_users)] \
    .merge(top_k_pop_items
        , on="book_id", how="left")

cold_user_items_no_avg_rating_idx = cold_users_events_with_recs["avg_rating"].isnull()
cold_user_recs = cold_users_events_with_recs[~cold_user_items_no_avg_rating_idx] \
    [["user_id", "book_id", "rating", "avg_rating"]]

In [13]:
cold_user_recs.shape[0]\
    /events_test[events_test["user_id"].isin(cold_users)].shape[0]

0.19768403639371382

In [14]:
1 - 0.1977

0.8023

In [16]:
# посчитаем метрики рекомендаций
from sklearn.metrics import mean_squared_error, mean_absolute_error

rmse = mean_squared_error(cold_user_recs["rating"], cold_user_recs["avg_rating"], squared=False)
mae = mean_absolute_error(cold_user_recs["rating"], cold_user_recs["avg_rating"])
print(round(rmse, 2), round(mae, 2)) 

0.78 0.62


In [17]:
# посчитаем покрытие холодных пользователей рекомендациями

cold_users_hit_ratio = cold_users_events_with_recs.groupby("user_id").agg(hits=("avg_rating", lambda x: (~x.isnull()).mean()))

print(f"Доля пользователей без релевантных рекомендаций: {(cold_users_hit_ratio == 0).mean().iat[0]:.2f}")
print(f"Среднее покрытие пользователей: {cold_users_hit_ratio[cold_users_hit_ratio != 0].mean().iat[0]:.2f}")

Доля пользователей без релевантных рекомендаций: 0.59
Среднее покрытие пользователей: 0.44


# === Знакомство: первые персональные рекомендации

In [18]:
events[['user_id', 'book_id', 'rating']].groupby(['user_id', 'book_id']).

,user_id,book_id,rating
0,1229132,22034,5
1,1229132,22318578,5
2,1229132,22551730,4
3,1229132,22816087,5
5,1229132,17910054,3
...,...,...,...
12914452,1364473,5297,5
12914453,1364473,4900,2
12914454,1364473,14836,3
12914456,1297020,10210,5


In [19]:
events['rating'].value_counts()

rating
4    4528257
5    3823215
3    2549892
2     667366
1     182356
Name: count, dtype: int64

In [20]:
events['user_id'].nunique()

430585

In [21]:
events['book_id'].nunique()

41673

In [22]:
events['book_id'].nunique() * events['user_id'].nunique()

17943768705

In [39]:
events.iloc[155:165].pivot(index='user_id', 
                        columns='book_id', 
                        values='rating')

book_id,16793,32071,6493208,10964693,13497818,17297487,18892134,22917292,25005214,25362018
user_id,,,,,,,,,,
1196635,4.0,3.0,4.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN
1418222,NaN,NaN,NaN,NaN,NaN,3.0,4.0,4.0,3.0,3.0


In [40]:
events.iloc[155:165].pivot(index='user_id', 
                        columns='book_id', 
                        values='rating').isna().sum().sum()

10

In [41]:
all_w = events.pivot(index='user_id', 
                        columns='book_id', 
                        values='rating')
all_w.isna().sum().sum() / (all_w.shape[0]*all_w.shape[1])

/tmp/ipykernel_3705/2920062743.py:1: PerformanceWarning: The following operation may generate 17943768705 cells in the resulting pandas object.
  all_w = events.pivot(index='user_id',


MemoryError: Unable to allocate 134. GiB for an array with shape (430585, 41673) and data type float64

In [47]:
1 - events[['user_id', 'book_id', 'rating']].groupby(['user_id', 'book_id']).rating.count().shape[0] \
 /(events['book_id'].nunique() * events['user_id'].nunique())

0.9993451160571009

In [49]:
from surprise import Dataset, Reader
from surprise import SVD

# используем Reader из библиотеки surprise для преобразования событий (events)
# в формат, необходимый surprise
reader = Reader(rating_scale=(1, 5))
surprise_train_set = Dataset.load_from_df(events_train[['user_id', 'book_id', 'rating']], reader)
surprise_train_set = surprise_train_set.build_full_trainset()

# инициализируем модель
svd_model = SVD(n_factors=100, random_state=0)

# обучаем модель
svd_model.fit(surprise_train_set)

In [50]:
surprise_test_set = list(
    events_test[['user_id', 'book_id', 'rating']].itertuples(index=False))

# получаем рекомендации для тестовой выборки
svd_predictions = svd_model.test(surprise_test_set)

In [51]:
from surprise import accuracy

rmse = accuracy.rmse(svd_predictions)
mae = accuracy.mae(svd_predictions)
                     
print(rmse, mae)

RMSE: 0.8289
MAE:  0.6474
0.8288711689059135 0.647437483750257


In [52]:
from surprise import NormalPredictor

# инициализируем состояние генератора, это необходимо для получения
# одной и той же последовательности случайных чисел, только в учебных целях
np.random.seed(0)

random_model = NormalPredictor()

random_model.fit(surprise_train_set)
random_predictions = random_model.test(surprise_test_set)

In [53]:
rmse = accuracy.rmse(random_predictions)
mae = accuracy.mae(random_predictions)
                     
print(rmse, mae)

RMSE: 1.2628
MAE:  1.0018
1.2628030301013033 1.0017726877569562


In [55]:
def get_recommendations_svd(user_id, events, model, include_seen=True, n=5):

    """ возвращает n рекомендаций для user_id """
    
    # получим список идентификаторов всех книг
    all_items = set(events['book_id'].unique())
        
    # учитываем флаг, стоит ли уже прочитанные книги включать в рекомендации
    if include_seen:
        items_to_predict = list(all_items)
    else:
        # получим список книг, которые пользователь уже прочитал ("видел")
        seen_items = set(events[events['user_id']==user_id]['book_id'].unique())
        
        # книги, которые пользователь ещё не читал
        # только их и будем включать в рекомендации
        items_to_predict = list(all_items - seen_items)
    
    # получаем скоры для списка книг, т. е. рекомендации
    predictions = [model.predict(user_id, book_id) for book_id in items_to_predict]
    
    # сортируем рекомендации по убыванию скора и берём только n первых
    recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:n]
    
    return pd.DataFrame([(pred.iid, pred.est) for pred in recommendations], columns=["book_id", "score"])

In [60]:
get_recommendations_svd(1296647, events_train, svd_model, include_seen=True, n=5)

,book_id,score
0,24812,5.000000
1,7864312,4.981188
2,25793186,4.912001
3,12174312,4.898052
4,13208,4.894869


In [64]:
# выберем произвольного пользователя из тренировочной выборки ("прошлого")
user_id = events_train['user_id'].sample().iat[0]

print(f"user_id: {user_id}")

print("История (последние события, recent)")
user_history = (
    events_train
    .query("user_id == @user_id")
    .merge(items.set_index("book_id")[["author", "title", "genre_and_votes"]], on="book_id")
)
user_history_to_print = user_history[["author", "title", "started_at", "read_at", "rating", "genre_and_votes"]].tail(10)
display(user_history_to_print)

print("Рекомендации")
user_recommendations = get_recommendations_svd(user_id, events_train, svd_model)
user_recommendations = user_recommendations.merge(items[["book_id", "author", "title", "genre_and_votes"]], on="book_id")
display(user_recommendations)

user_id: 1349218
История (последние события, recent)


,author,title,started_at,read_at,rating,genre_and_votes
53,Kimberly Derting,"The Body Finder (The Body Finder, #1)",2011-03-19,2011-03-20,4,"{'Young Adult': 1320, 'Fantasy-Paranormal': 93..."
54,April Lindner,Jane,2011-06-03,2011-06-06,5,"{'Young Adult': 434, 'Romance': 340, 'Retellin..."
55,Cynthia Hand,"Unearthly (Unearthly, #1)",2011-05-18,2011-05-24,5,"{'Young Adult': 2318, 'Fantasy': 1933, 'Parano..."
56,Cassandra Clare,"Clockwork Prince (The Infernal Devices, #2)",2012-01-21,2012-02-05,5,"{'Fantasy': 7212, 'Young Adult': 5170, 'Romanc..."
57,"Orson Scott Card, Stefan Rudnicki, Harlan Ellison","Ender's Game (Ender's Saga, #1)",2011-10-07,2011-10-21,5,"{'Science Fiction': 15315, 'Fiction': 7488, 'Y..."
58,Suzanne Collins,"The Hunger Games (The Hunger Games, #1)",2011-11-20,2011-11-20,5,"{'Young Adult': 30042, 'Fiction': 16754, 'Scie..."
59,Ally Condie,"Matched (Matched, #1)",2011-12-14,2011-12-30,4,"{'Young Adult': 7846, 'Science Fiction-Dystopi..."
60,Alex Flinn,"Beastly (Beastly, #1; Kendra Chronicles, #1)",2011-03-11,2011-03-11,4,"{'Young Adult': 2672, 'Fantasy': 2570, 'Romanc..."
61,Stephenie Meyer,"The Host (The Host, #1)",2011-03-30,2011-04-02,5,"{'Young Adult': 4380, 'Science Fiction': 4168,..."
62,Sara Gruen,Water for Elephants,2011-04-28,2011-05-03,5,"{'Fiction': 9703, 'Historical-Historical Ficti..."


Рекомендации


,book_id,score,author,title,genre_and_votes
0,1,5,J.K. Rowling,Harry Potter and the Half-Blood Prince (Harry ...,"{'Fantasy': 46400, 'Young Adult': 15083, 'Fict..."
1,2,5,"J.K. Rowling, Mary GrandPré",Harry Potter and the Order of the Phoenix (Har...,"{'Fantasy': 46485, 'Young Adult': 15194, 'Fict..."
2,3,5,"J.K. Rowling, Mary GrandPré",Harry Potter and the Sorcerer's Stone (Harry P...,"{'Fantasy': 59818, 'Fiction': 17918, 'Young Ad..."
3,5,5,"J.K. Rowling, Mary GrandPré",Harry Potter and the Prisoner of Azkaban (Harr...,"{'Fantasy': 49784, 'Young Adult': 15393, 'Fict..."
4,6,5,"J.K. Rowling, Mary GrandPré",Harry Potter and the Goblet of Fire (Harry Pot...,"{'Fantasy': 48257, 'Young Adult': 15483, 'Fict..."


# === Базовые подходы: коллаборативная фильтрация

In [9]:
import scipy
import sklearn.preprocessing

# перекодируем идентификаторы пользователей: 
# из имеющихся в последовательность 0, 1, 2, ...
user_encoder = sklearn.preprocessing.LabelEncoder()
user_encoder.fit(events["user_id"])
events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])

# перекодируем идентификаторы объектов: 
# из имеющихся в последовательность 0, 1, 2, ...
item_encoder = sklearn.preprocessing.LabelEncoder()
item_encoder.fit(items["book_id"])
items["item_id_enc"] = item_encoder.transform(items["book_id"])
events_train["item_id_enc"] = item_encoder.transform(events_train["book_id"])
events_test["item_id_enc"] = item_encoder.transform(events_test["book_id"])

/tmp/ipykernel_2041/1879297858.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
/tmp/ipykernel_2041/1879297858.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])
/tmp/ipykernel_2041/1879297858.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

In [11]:
events_train['item_id_enc'].max()

43304

In [12]:
(events['book_id'].nunique() * events['user_id'].nunique()) / 1024**3

16.71143686864525

In [13]:
# создаём sparse-матрицу формата CSR 
user_item_matrix_train = scipy.sparse.csr_matrix((
    events_train["rating"],
    (events_train['user_id_enc'], events_train['item_id_enc'])),
    dtype=np.int8)

In [14]:
import sys

sum([sys.getsizeof(i) for i in user_item_matrix_train.data])/1024**3

0.26370687410235405

In [15]:
from implicit.als import AlternatingLeastSquares

als_model = AlternatingLeastSquares(factors=50, iterations=50, regularization=0.05, random_state=0)
als_model.fit(user_item_matrix_train)

/home/mle-user/mle_projects/rec_sys_2/env_recsys_start/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mle-user/mle_projects/rec_sys_2/env_recsys_start/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 4 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|██████████| 50/50 [03:03<00:00,  3.67s/it]


In [16]:
def get_recommendations_als(user_item_matrix, model, user_id, user_encoder, item_encoder, include_seen=True, n=5):
    """
    Возвращает отранжированные рекомендации для заданного пользователя
    """
    user_id_enc = user_encoder.transform([user_id])[0]
    recommendations = model.recommend(
         user_id_enc, 
         user_item_matrix[user_id_enc], 
         filter_already_liked_items=not include_seen,
         N=n)
    recommendations = pd.DataFrame({"item_id_enc": recommendations[0], "score": recommendations[1]})
    recommendations["item_id"] = item_encoder.inverse_transform(recommendations["item_id_enc"])
    
    return recommendations

In [17]:
# получаем список всех возможных user_id (перекодированных)
user_ids_encoded = range(len(user_encoder.classes_))

# получаем рекомендации для всех пользователей
als_recommendations = als_model.recommend(
    user_ids_encoded, 
    user_item_matrix_train[user_ids_encoded], 
    filter_already_liked_items=False, N=100)

In [18]:
# преобразуем полученные рекомендации в табличный формат
item_ids_enc = als_recommendations[0]
als_scores = als_recommendations[1]

als_recommendations = pd.DataFrame({
    "user_id_enc": user_ids_encoded,
    "item_id_enc": item_ids_enc.tolist(), 
    "score": als_scores.tolist()})
als_recommendations = als_recommendations.explode(["item_id_enc", "score"], ignore_index=True)

# приводим типы данных
als_recommendations["item_id_enc"] = als_recommendations["item_id_enc"].astype("int")
als_recommendations["score"] = als_recommendations["score"].astype("float")

# получаем изначальные идентификаторы
als_recommendations["user_id"] = user_encoder.inverse_transform(als_recommendations["user_id_enc"])
als_recommendations["item_id"] = item_encoder.inverse_transform(als_recommendations["item_id_enc"])
als_recommendations = als_recommendations.drop(columns=["user_id_enc", "item_id_enc"])

In [19]:
als_recommendations = als_recommendations[["user_id", "item_id", "score"]]
als_recommendations.to_parquet("als_recommendations.parquet")

In [21]:
als_recommendations.columns = ["user_id", "book_id", "score"]

In [22]:
als_recommendations = (
    als_recommendations
    .merge(events_test[["user_id", "book_id", "rating"]]
               .rename(columns={"rating": "rating_test"}), 
           on=["user_id", "book_id"], how="left")
)

In [26]:
als_recommendations['rating_test'].value_counts(dropna=False)

rating_test
NaN    42978006
5.0       34710
4.0       29713
3.0       12687
2.0        2743
1.0         641
Name: count, dtype: int64

In [27]:
import sklearn.metrics

def compute_ndcg(rating: pd.Series, score: pd.Series, k):

    """ подсчёт ndcg
    rating: истинные оценки
    score: оценки модели
    k: количество айтемов (по убыванию score) для оценки, остальные - отбрасываются
    """
    
    # если кол-во объектов меньше 2, то NDCG - не определена
    if len(rating) < 2:
        return np.nan

    ndcg = sklearn.metrics.ndcg_score(np.asarray([rating.to_numpy()]), 
                                      np.asarray([score.to_numpy()]), k=k)

    return ndcg

In [28]:
rating_test_idx = ~als_recommendations["rating_test"].isnull()
ndcg_at_5_scores = als_recommendations[rating_test_idx].groupby("user_id").apply(
    lambda x: compute_ndcg(x["rating_test"], x["score"], k=5))

In [29]:
print(ndcg_at_5_scores.mean())

0.975946709792109


# === Базовые подходы: контентные рекомендации

In [30]:
# cделали dict из str используя встроенную функцию питона
items["genre_and_votes"] = items["genre_and_votes"].apply(eval)

In [34]:
type(items["genre_and_votes"].iloc[0])

dict

In [33]:
items["genre_and_votes"].iloc[:10]

3      {'Womens Fiction-Chick Lit': 739, 'Fiction': 442}
6                            {'Politics': 1, 'Humor': 1}
15     {'Christian': 395, 'Nonfiction': 392, 'Religio...
16     {'Christian': 225, 'Religion-Theology': 154, '...
17     {'Historical-Historical Fiction': 284, 'Childr...
141    {'Fiction': 192, 'Mystery': 174, 'Thriller': 8...
309    {'Science Fiction': 80, 'Fantasy': 55, 'Fictio...
429    {'Fantasy': 493, 'Childrens-Middle Grade': 259...
440    {'Fiction': 7, 'Health-Mental Health': 2, 'Par...
441    {'Autobiography-Memoir': 49, 'Nonfiction': 32,...
Name: genre_and_votes, dtype: object

In [37]:
for k, v, in items.iloc[:3].iterrows():
    print(v['genre_and_votes'])

{'Womens Fiction-Chick Lit': 739, 'Fiction': 442}
{'Politics': 1, 'Humor': 1}
{'Christian': 395, 'Nonfiction': 392, 'Religion': 142, 'Spirituality': 104, 'Biography': 96, 'Autobiography-Memoir': 93, 'Inspirational': 87, 'Religion-Christianity': 73, 'Biography-Autobiography': 64, 'Religion-Faith': 54}


In [38]:
def get_genres(items):

    """ 
    извлекает список жанров по всем книгам, 
    подсчитывает долю голосов по каждому их них
    """
    
    genres_counter = {}
    
    for k, v, in items.iterrows():
        genre_and_votes = v['genre_and_votes']
        if genre_and_votes is None or not isinstance(genre_and_votes, dict):
            continue
        for genre, votes in genre_and_votes.items():
            # увеличиваем счётчик жанров
            try:
                genres_counter[genre] += votes
            except KeyError:
                genres_counter[genre] = 0

    genres = pd.Series(genres_counter, name="votes")
    genres = genres.to_frame()
    genres = genres.reset_index().rename(columns={"index": "name"})
    genres.index.name = "genre_id"
    
    return genres
   
genres = get_genres(items)

In [39]:
genres["score"] = genres["votes"] / genres["votes"].sum()
genres.sort_values(by="score", ascending=False).head(10)

,name,votes,score
genre_id,,,
25,Fantasy,6850060,0.149651
1,Fiction,6406256,0.139955
38,Classics,3414934,0.074605
18,Young Adult,3296951,0.072027
34,Romance,2422614,0.052926
5,Nonfiction,1737406,0.037957
16,Historical-Historical Fiction,1531205,0.033452
20,Mystery,1371196,0.029956
24,Science Fiction,1218917,0.026629


In [40]:
def get_item2genre_matrix(genres, items):

    genre_names_to_id = genres.reset_index().set_index("name")["genre_id"].to_dict()
    
    # list to build CSR matrix
    genres_csr_data = []
    genres_csr_row_idx = []
    genres_csr_col_idx = []
    
    for item_idx, (k, v) in enumerate(items.iterrows()):
        if v["genre_and_votes"] is None:
            continue
        for genre_name, votes in v["genre_and_votes"].items():
            genre_idx = genre_names_to_id[genre_name]
            genres_csr_data.append(int(votes))
            genres_csr_row_idx.append(item_idx)
            genres_csr_col_idx.append(genre_idx)

    genres_csr = scipy.sparse.csr_matrix((genres_csr_data, 
                                          (genres_csr_row_idx, genres_csr_col_idx)), 
                                          shape=(len(items), len(genres)))
    # нормализуем, чтобы сумма оценок принадлежности к жанру была равна 1
    genres_csr = sklearn.preprocessing.normalize(genres_csr, norm='l1', axis=1)
    
    return genres_csr

In [41]:
items = items.sort_values(by="item_id_enc")
all_items_genres_csr = get_item2genre_matrix(genres, items)

In [51]:
genres

,name,votes,score
genre_id,,,
0,Womens Fiction-Chick Lit,254558,0.005561
1,Fiction,6406256,0.139955
2,Politics,103296,0.002257
3,Humor,304302,0.006648
4,Christian,105273,0.002300
...,...,...,...
810,German History-Nazi Party,0,0.000000
811,Favorites,0,0.000000
812,History-Latin American History,0,0.000000


In [53]:
user_id = 1000010
user_events = events_train.query("user_id == @user_id")[["book_id", "rating"]]
user_items = items[items["book_id"].isin(user_events["book_id"])]

user_items_genres_csr = get_item2genre_matrix(genres,user_items)
user_items_genres_csr

<22x815 sparse matrix of type '<class 'numpy.float64'>'
	with 149 stored elements in Compressed Sparse Row format>

In [54]:
# вычислим склонность пользователя к жанрам как среднее взвешенное значение популяции на его оценки книг.

# преобразуем пользовательские оценки из списка в вектор-столбец
user_ratings = user_events["rating"].to_numpy() / 5
user_ratings = np.expand_dims(user_ratings, axis=1)

user_items_genres_weighted = user_items_genres_csr.multiply(user_ratings)

user_genres_scores = np.asarray(user_items_genres_weighted.mean(axis=0))

In [55]:
# выведем список жанров, которые предпочитает пользователь

user_genres = genres.copy()
user_genres["score"] = np.ravel(user_genres_scores)
user_genres = user_genres[user_genres["score"] > 0].sort_values(by=["score"], ascending=False)

user_genres.head(5)

,name,votes,score
genre_id,,,
1,Fiction,6406256,0.185241
38,Classics,3414934,0.103879
25,Fantasy,6850060,0.072447
5,Nonfiction,1737406,0.050865
24,Science Fiction,1218917,0.040920


In [65]:
from sklearn.metrics.pairwise import cosine_similarity

# вычисляем сходство между вектором пользователя и векторами по книгам
similarity_scores = cosine_similarity(all_items_genres_csr, user_genres_scores)

# преобразуем в одномерный массив
similarity_scores = similarity_scores.flatten()

# получаем индексы top-k (по убыванию значений), по сути, индексы книг (encoded)
k = 5
top_k_indices = np.argsort(similarity_scores)[-k:]

In [66]:
top_k_indices

array([ 4460,  9476, 14087, 36093,  4471])

In [67]:
similarity_scores[4460]

0.9125284120433571

In [68]:
selected_items = items[items["item_id_enc"].isin(top_k_indices)]

with pd.option_context("max_colwidth", 100):
   display(selected_items[["author", "title", "genre_and_votes"]])

,author,title,genre_and_votes
80465,G.K. Chesterton,The Napoleon of Notting Hill,"{'Fiction': 166, 'Classics': 88, 'Fantasy': 44, 'Humor': 22, 'Literature': 20}"
1168335,Ray Bradbury,"Dandelion Wine (Green Town, #1)","{'Fiction': 1438, 'Classics': 914, 'Science Fiction': 529, 'Fantasy': 456, 'Young Adult': 212}"
393210,"G.K. Chesterton, Jonathan Lethem",The Man Who Was Thursday: A Nightmare,"{'Fiction': 1257, 'Classics': 929, 'Mystery': 469, 'Fantasy': 293, 'Philosophy': 156, 'Literatur..."
2244467,Samuel Butler,"Erewhon (Erewhon , #1)","{'Fiction': 162, 'Classics': 139, 'Science Fiction': 60, 'Fantasy': 55}"
39408,"Paulo Coelho, Alan R. Clarke, James Noel Smith",The Alchemist,"{'Fiction': 14023, 'Classics': 5787, 'Fantasy': 3289, 'Philosophy': 2759}"


# === Базовые подходы: валидация

In [71]:
def process_events_recs_for_binary_metrics(events_train, events_test, recs, top_k=None):

    """
    размечает пары <user_id, item_id> для общего множества пользователей признаками
    - gt (ground truth)
    - pr (prediction)
    top_k: расчёт ведётся только для top k-рекомендаций
    """

    events_test["gt"] = True
    common_users = set(events_test["user_id"]) & set(recs["user_id"])

    print(f"Common users: {len(common_users)}")
    
    events_for_common_users = events_test[events_test["user_id"].isin(common_users)].copy()
    recs_for_common_users = recs[recs["user_id"].isin(common_users)].copy()

    recs_for_common_users = recs_for_common_users.sort_values(["user_id", "score"], ascending=[True, False])

    # оставляет только те item_id, которые были в events_train, 
    # т. к. модель не имела никакой возможности давать рекомендации для новых айтемов
    events_for_common_users = events_for_common_users[events_for_common_users["book_id"].isin(events_train["book_id"].unique())]

    if top_k is not None:
        recs_for_common_users = recs_for_common_users.groupby("user_id").head(top_k)
    
    events_recs_common = events_for_common_users[["user_id", "book_id", "gt"]].merge(
        recs_for_common_users[["user_id", "book_id", "score"]], 
        on=["user_id", "book_id"], how="outer")    

    events_recs_common["gt"] = events_recs_common["gt"].fillna(False)
    events_recs_common["pr"] = ~events_recs_common["score"].isnull()
    
    events_recs_common["tp"] = events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fp"] = ~events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fn"] = events_recs_common["gt"] & ~events_recs_common["pr"]

    return events_recs_common

In [72]:
events_recs_for_binary_metrics = process_events_recs_for_binary_metrics(
  events_train,
    events_test, 
    als_recommendations, 
    top_k=5)

/tmp/ipykernel_2041/4078253895.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["gt"] = True


Common users: 123223


In [73]:
def compute_cls_metrics(events_recs_for_binary_metric):
    
    groupper = events_recs_for_binary_metric.groupby("user_id")

    # precision = tp / (tp + fp)
    precision = groupper["tp"].sum()/(groupper["tp"].sum()+groupper["fp"].sum())
    precision = precision.fillna(0).mean()
    
    # recall = tp / (tp + fn)
    recall = groupper["tp"].sum()/(groupper["tp"].sum()+groupper["fn"].sum())
    recall = recall.fillna(0).mean()

    return precision, recall

In [74]:
compute_cls_metrics(events_recs_for_binary_metrics)

(0.007581376853347184, 0.014121568795222568)

# === Двухстадийный подход: метрики

In [75]:
als_recommendations

,user_id,book_id,score,rating_test
0,1000000,3,0.990941,NaN
1,1000000,15881,0.896617,NaN
2,1000000,5,0.864404,NaN
3,1000000,6,0.822254,NaN
4,1000000,2,0.774095,NaN
...,...,...,...,...
43058495,1430584,13206900,0.096082,NaN
43058496,1430584,5060378,0.096065,NaN
43058497,1430584,16071764,0.094949,NaN
43058498,1430584,9969571,0.094927,NaN


In [84]:
# расчёт покрытия по объектам
all = items.book_id.unique().tolist()
recs = als_recommendations.book_id.unique().tolist()
cov_items = len(list(set(recs)&set(all)))/len(all)
print(f"{cov_items:.2f}")

0.09


In [ ]:
# разметим каждую рекомендацию признаком read
events_train["read"] = True
als_recommendations = als_recommendations.merge(events_train,
    on=["user_id", "book_id"], how="left")
als_recommendations["read"] = als_recommendations["read"].fillna(False).astype("bool")

In [86]:
# проставим ранги
als_recommendations = als_recommendations.sort_values('score', ascending=False)
als_recommendations["rank"] = als_recommendations.groupby("user_id").cumcount() + 1

# посчитаем novelty по пользователям
novelty_5 = (1-als_recommendations.query("rank <= 5").groupby("user_id")["read"].mean())

In [88]:
# посчитаем средний novelty
novelty_5.mean()

0.607333279143491

# === Двухстадийный подход: модель

In [89]:
# задаём точку разбиения
split_date_for_labels = pd.to_datetime("2017-09-15").date()

split_date_for_labels_idx = events_test["started_at"] < split_date_for_labels
events_labels = events_test[split_date_for_labels_idx].copy()
events_test_2 = events_test[~split_date_for_labels_idx].copy()

In [90]:
events_labels['user_id'].nunique()

99849

In [92]:
# загружаем рекомендации от двух базовых генераторов
als_recommendations = pd.read_parquet("candidates/training/als_recommendations.parquet")
content_recommendations = pd.read_parquet("candidates/training/content_recommendations.parquet")

candidates = pd.merge(
    als_recommendations[["user_id", "item_id", "score"]].rename(columns={"score": "als_score"}),
    content_recommendations[["user_id", "item_id", "score"]].rename(columns={"score": "cnt_score"}),
    on=['user_id','item_id'],
    how="outer")

In [93]:
candidates.shape[0]

82993094

In [98]:
# добавляем таргет к кандидатам со значением:
# — 1 для тех item_id, которые пользователь прочитал
# — 0, для всех остальных 

events_labels["target"] = 1
candidates = candidates.merge(events_labels.rename(columns={'book_id':'item_id'})[["user_id", "item_id", "target"]], 
                              on=["user_id", "item_id"],how='left')
candidates["target"] = candidates["target"].fillna(0).astype("int")

# в кандидатах оставляем только тех пользователей, у которых есть хотя бы один положительный таргет
candidates_to_sample = candidates.groupby("user_id").filter(lambda x: x["target"].sum() > 0)

# для каждого пользователя оставляем только 4 негативных примера
negatives_per_user = 4
candidates_for_train = pd.concat([
    candidates_to_sample.query("target == 1"),
    candidates_to_sample.query("target == 0") \
        .groupby("user_id") \
        .apply(lambda x: x.sample(negatives_per_user, random_state=0))
    ])

In [99]:
candidates_for_train.shape[0]

213708

In [100]:
from catboost import CatBoostClassifier, Pool

# задаём имена колонок признаков и таргета
features = ['als_score', 'cnt_score']
target = 'target'

# Create the Pool object
train_data = Pool(
    data=candidates_for_train[features], 
    label=candidates_for_train[target])

# инициализируем модель CatBoostClassifier
cb_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    loss_function='Logloss',
    verbose=100,
    random_seed=0
)

# тренируем модель
cb_model.fit(train_data)

0:	learn: 0.6526057	total: 73.9ms	remaining: 1m 13s
100:	learn: 0.5118959	total: 1.84s	remaining: 16.4s
200:	learn: 0.5111710	total: 3.68s	remaining: 14.6s
300:	learn: 0.5105208	total: 5.54s	remaining: 12.9s
400:	learn: 0.5100174	total: 7.36s	remaining: 11s
500:	learn: 0.5095747	total: 9.21s	remaining: 9.17s
600:	learn: 0.5091600	total: 11s	remaining: 7.32s
700:	learn: 0.5087803	total: 12.9s	remaining: 5.5s
800:	learn: 0.5084220	total: 14.7s	remaining: 3.65s
900:	learn: 0.5080930	total: 16.5s	remaining: 1.82s
999:	learn: 0.5078081	total: 18.3s	remaining: 0us


In [104]:
# загружаем рекомендации от двух базовых генераторов
als_recommendations_2 = pd.read_parquet("candidates/inference/als_recommendations.parquet")
content_recommendations_2 = pd.read_parquet("candidates/inference/content_recommendations.parquet")

candidates_to_rank = pd.merge(
    als_recommendations_2[["user_id", "item_id", "score"]].rename(columns={"score": "als_score"}),
    content_recommendations_2[["user_id", "item_id", "score"]].rename(columns={"score": "cnt_score"}),
    on=['user_id','item_id'],
    how="outer")

# оставляем только тех пользователей, что есть в тестовой выборке, для экономии ресурсов
candidates_to_rank = candidates_to_rank[candidates_to_rank["user_id"].isin(events_test_2["user_id"].drop_duplicates())]
print(len(candidates_to_rank))

14517152


In [105]:
inference_data = Pool(data=candidates_to_rank[features])
predictions = cb_model.predict_proba(inference_data)

candidates_to_rank["cb_score"] = predictions[:, 1]

# для каждого пользователя проставляем rank, начиная с 1 — это максимальный cb_score
candidates_to_rank = candidates_to_rank.sort_values(["user_id", "cb_score"], ascending=[True, False])
candidates_to_rank["rank"] = 1
candidates_to_rank["rank"] = candidates_to_rank.groupby("user_id")["rank"].cumsum()
candidates_to_rank

,user_id,item_id,als_score,cnt_score,cb_score,rank
347,1000003,49628,0.446143,0.906649,0.583617,1
300,1000003,7260188,1.129979,NaN,0.509005,2
301,1000003,6148028,1.123475,NaN,0.509005,3
302,1000003,2767052,1.112699,NaN,0.509005,4
320,1000003,43641,0.617602,NaN,0.477032,5
...,...,...,...,...,...,...
82993814,1430580,183087,NaN,0.981247,0.066960,187
82993807,1430580,33906,NaN,0.988908,0.064737,188
82993808,1430580,82436,NaN,0.987934,0.064737,189
82993809,1430580,82780,NaN,0.987217,0.061955,190


In [106]:
max_recommendations_per_user = 100
final_recommendations = candidates_to_rank.query('rank<=100')

In [107]:
final_recommendations.shape[0]

7519400

In [114]:
events_inference = pd.concat([events_train, events_labels])

cb_events_recs_for_binary_metrics_5 = process_events_recs_for_binary_metrics(
    events_inference,
    events_test_2,
    final_recommendations.rename(columns={"cb_score": "score", 
                                          'item_id':'book_id'}), 
    top_k=5)

cb_precision_5, cb_recall_5 = compute_cls_metrics(cb_events_recs_for_binary_metrics_5)

print(f"precision: {cb_precision_5:.3f}, recall: {cb_recall_5:.3f}")

Common users: 75194
precision: 0.006, recall: 0.015


# === Двухстадийный подход: построение признаков